In [13]:
from __future__ import print_function                  # Allows for python3 printing
import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, SimpleRNN,GRU,LSTM
from keras.callbacks import ModelCheckpoint
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras import optimizers
import numpy as np
import sys

In [14]:
filename = "sonnet.txt"
#read the file and make all chracaters lowercase
text = open(filename).read().lower() 

In [15]:
# summarize the loaded data
print('text length:', len(text))    # this prints the length of the text 

chars = sorted(list(set(text)))    # sorted():Return a new list containing all items from the iterable in ascending order.
print('total chars:', len(chars))
print(chars)
##This should print 38.

text length: 95690
total chars: 38
['\n', ' ', '!', "'", '(', ')', ',', '-', '.', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [16]:
# 4.1 encoding and decoding dictionaries
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [17]:
# 4.2 cut corpus into equal length sequences
maxlen = 40
step = 3

sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])               
    next_chars.append(text[i + maxlen])

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)   #generate zeros with size[number of sentences,40,38]
y = np.zeros((len(sentences), len(chars)), dtype=np.bool) #generate zeros with size[number of sentences,38]
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

# 备注
这里我尝试了LSTM和GRU，发现使用GRU明显能得到更低的loss。之后我尝试着换用了另外的几个optimizer，其中Nadam使loss较之前有了极大的改善，其输出结果如下：

In [34]:
model = Sequential()
model.add(GRU(128, input_shape=(x.shape[1], x.shape[2])))        #using GRU instead of RNN
model.add(Dense(len(chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Nadam())   #changed the optimizer to Nadam 

In [35]:
for iteration in range(1, 11):
    model.fit(x, y,
              batch_size=128,
              epochs=10)
    
# generating text
    print('\nIteration', iteration)
    start_index = np.random.randint(0, len(text) - maxlen - 1)
    sentence = text[start_index: start_index + maxlen]


    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]      #predict using model, generating a matrix of softmax
        next_index = np.argmax(preds)                    #we want the index with highest probability 
        next_char = indices_char[next_index]             #convert number back to character using the dictionary
        sentence = sentence[1:] + next_char              #append the character predicted to the sentence (start from the second character)

        sys.stdout.write(next_char)                
        sys.stdout.flush()
    print('\n')

Epoch 1/10
31884/31884 [==============================] - 23s 725us/step - loss: 2.6146
Epoch 2/10
31884/31884 [==============================] - 28s 884us/step - loss: 2.1704
Epoch 3/10
31884/31884 [==============================] - 29s 902us/step - loss: 2.0123
Epoch 4/10
31884/31884 [==============================] - 30s 943us/step - loss: 1.9044
Epoch 5/10
31884/31884 [==============================] - 32s 1ms/step - loss: 1.8216
Epoch 6/10
31884/31884 [==============================] - 35s 1ms/step - loss: 1.7533
Epoch 7/10
31884/31884 [==============================] - 38s 1ms/step - loss: 1.6925
Epoch 8/10
31884/31884 [==============================] - 39s 1ms/step - loss: 1.6383
Epoch 9/10
31884/31884 [==============================] - 39s 1ms/step - loss: 1.5838
Epoch 10/10
31884/31884 [==============================] - 40s 1ms/step - loss: 1.5327

Iteration 1
 but in my must be thee,
when in the will he on the will me all my be,
  and the will he on the will my be the with mi

31884/31884 [==============================] - 37s 1ms/step - loss: 0.4630
Epoch 8/10
31884/31884 [==============================] - 42s 1ms/step - loss: 0.4257
Epoch 9/10
31884/31884 [==============================] - 49s 2ms/step - loss: 0.4156
Epoch 10/10
31884/31884 [==============================] - 50s 2ms/step - loss: 0.4105: 2s

Iteration 7
eaph and love, where in their race,
and caplangion queppeainded, not so to see,
like the sweet her thou dost bide i wan wrich ho reathes
asted bestidaity how my sees in my rend
fart thy heart, and this wills the world'd,
and they all me at fllater belies thou ures
 for that gendsus' eccoprare when thou my selfen'?
whise have you mand, make he waskin gond,
which thou dost bern is be steno summer gr

Epoch 1/10
31884/31884 [==============================] - 46s 1ms/step - loss: 0.4442
Epoch 2/10
31884/31884 [==============================] - 46s 1ms/step - loss: 0.4232
Epoch 3/10
31884/31884 [==============================] - 49s 2ms/step - lo